In [15]:
import  pyodbc

import pandas as pd



### Connection to your SQL Database

In [5]:

# Connection parameters
server = input('Your server name')
database =input('Your version of avdentureworks')


# Establish connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')

# Create a cursor and fetch data from 'production.product'
cursor = conn.cursor()
cursor.execute("SELECT * FROM production.product")
rows = cursor.fetchall()

# Convert data to a DataFrame for 'production.product'
products = pd.DataFrame([tuple(row) for row in rows], columns=[column[0] for column in cursor.description])

# Create another cursor and fetch data from 'sales.salesorderdetail'
cursor.execute("SELECT * FROM sales.salesorderdetail")
rows = cursor.fetchall()

# Convert data to a DataFrame for 'sales.salesorderdetail'
sales_order_detail = pd.DataFrame([tuple(row) for row in rows], columns=[column[0] for column in cursor.description])

# Create another cursor and fetch data from 'VendorProduct'
cursor.execute("SELECT * FROM purchasing.ProductVendor")
rows = cursor.fetchall()
# Convert data to a DataFrame for 'purchasing.ProductVendor'
ProductVendor = pd.DataFrame([tuple(row) for row in rows], columns=[column[0] for column in cursor.description])
# Close the cursor and connection
cursor.close()
conn.close()




### Create the finished_goods table and the abc_analysis

In [36]:



# Filtering to include only finished goods
finished_goods = products[products['FinishedGoodsFlag'] == 1]

# Merging the Product and SalesOrderDetail tables on ProductID
sales_data = pd.merge(finished_goods, sales_order_detail, on='ProductID', how='left').fillna(0)

# Grouping the data by ProductID and Name and calculating sales totals
abc = sales_data.groupby(['ProductID', 'Name']).agg({'LineTotal': 'sum'}).reset_index().sort_values(by='LineTotal',ascending= False)
abc['TotalSales'] = abc['LineTotal'].sum()
abc['RunningTotal'] = abc['LineTotal'].cumsum()
abc['RunningPct'] = (abc['RunningTotal'] / abc['TotalSales']) * 100

# Assigning ABC categories
def categorize_abc(row):
    if row['RunningPct'] <= 80:
        return 'A'
    elif row['RunningPct'] > 80 and row['RunningPct'] <= 95:
        return 'B'
    else:
        return 'C'

abc['Category'] = abc.apply(categorize_abc, axis=1)

#merge with vendor
final_df= pd.merge(abc,ProductVendor[['ProductID','BusinessEntityID']],on= 'ProductID', how='left',)


final_df['BusinessEntityID']=final_df['BusinessEntityID'].fillna('Inhouse')

def categorize_procurement(row):
    if row['BusinessEntityID'] == 'Inhouse':
        return 'Inhouse'
    else:
        return 'Sourced'
final_df['Procuremnt'] = final_df.apply(categorize_procurement, axis=1)
# Display the final table
final_df




C:\Users\anast\AppData\Local\Temp\ipykernel_3760\2473883732.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sales_data = pd.merge(finished_goods, sales_order_detail, on='ProductID', how='left').fillna(0)


,ProductID,Name,LineTotal,TotalSales,RunningTotal,RunningPct,Category,BusinessEntityID,Procuremnt
0,782,"Mountain-200 Black, 38",4400592.800400,109846381.399888,4400592.800400,4.006133606149439229173267397,A,Inhouse,Inhouse
1,783,"Mountain-200 Black, 42",4009494.761841,109846381.399888,8410087.562241,7.656226318120275351625096599,A,Inhouse,Inhouse
2,779,"Mountain-200 Silver, 38",3693678.025272,109846381.399888,12103765.587513,11.01881139211140283293771275,A,Inhouse,Inhouse
3,780,"Mountain-200 Silver, 42",3438478.860423,109846381.399888,15542244.447936,14.14907277769629556637449675,A,Inhouse,Inhouse
4,781,"Mountain-200 Silver, 46",3434256.941928,109846381.399888,18976501.389864,17.27549068801946765773986780,A,Inhouse,Inhouse
...,...,...,...,...,...,...,...,...,...
310,818,LL Road Front Wheel,0,109846381.399888,109846381.399888,100,C,Inhouse,Inhouse
311,812,ML Road Handlebars,0,109846381.399888,109846381.399888,100,C,Inhouse,Inhouse
312,803,ML Fork,0,109846381.399888,109846381.399888,100,C,Inhouse,Inhouse
313,746,"HL Mountain Frame - Black, 46",0,109846381.399888,109846381.399888,100,C,Inhouse,Inhouse


### Insights

In [44]:
#Group the dataframe by category ,add the linetotal, count the number of product. for each category we will also find the percentage of product sourced and manufactured inhouse
sales_by_category= final_df.groupby(by='Category').agg({'LineTotal': 'sum','ProductID':'count','Procuremnt': lambda x: (x.value_counts(normalize=True) * 100).round(2).to_dict() })
sales_by_category

,LineTotal,ProductID,Procuremnt
Category,,,
A,87741348.821837,63,{'Inhouse': 100.0}
B,16529057.175720,66,"{'Inhouse': 87.88, 'Sourced': 12.12}"
C,5907150.032579,186,"{'Sourced': 51.08, 'Inhouse': 48.92}"
